In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import os
import cv2
import numpy as np 	
from skimage import io
import skimage.morphology as morph
import skimage.color as color
import skimage.measure as measure
import skimage.feature as feature
import skimage.util as util
from sklearn.feature_extraction.image import extract_patches_2d 
from scipy.fftpack import dctn
from scipy.fftpack import idctn

import matplotlib.pyplot as plt
%matplotlib inline

# Define paths to data folder and the processed data folder
data_path = os.path.join(".", "Data")
data_processed_path = os.path.join(".", "Processed")

In [2]:
# Find the max tick-bar distance in pixels

tick_bar_dist = []

for img_file in sorted(os.listdir(data_path)):
    
    # Ignore image files that aren't .jpg files (images)
    if img_file[-4:] != '.tif':
        continue
    
    # Store path for current image
    path_img = os.path.join(data_path, img_file)
    
    # Read image file
    img = cv2.imread(path_img)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Threshold image to form a binary image
    # Label the binary image and retain (crop image) the connected component with largest area
    
    ret,thresh = cv2.threshold(gray, 0, 255, 0)
    im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, 
                                                                        cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(c) for c in contours]
    max_index = np.argmax(areas)
    cnt=contours[max_index]
    x, y, w, h = cv2.boundingRect(cnt)
    crop_img_for_tick_bar = gray[y:y+h, x+20:x+w+40]
    
    # Crop the right 80% of the image to detect tick bar
    crop_img_h, crop_img_w = crop_img_for_tick_bar.shape
    img_tick = crop_img_for_tick_bar[:, int(0.7*crop_img_w):crop_img_w]
    
    # Detect the column containing tick bar by estimating intensity peaks
    img_tick_h, img_tick_w = img_tick.shape
    peaks = []

    for i in range(0, img_tick_w-1, 1):
        col = img_tick[:, i]
        peak_pts = feature.peak.peak_local_max(col, min_distance=10, threshold_abs=250, num_peaks=100)
        if peak_pts.shape[0] > 10 and peak_pts.shape[0] < 25:
            peaks = peak_pts
            column = i
            break
            
    # Measure the tick bar distance in pixels
    tick_distance_px = np.max(peaks) - np.min(peaks)
    
    # Store the tick bar distance
    tick_bar_dist.append(tick_distance_px)

# Calculate the maximum tick bar distance
max_tick_bar_dist_px = np.max(tick_bar_dist)

In [3]:
# Process all images

i = 0
for img_file in sorted(os.listdir(data_path)):
    # Ignore image files that aren't .jpg files (images)
    if img_file[-4:] != '.tif':
        continue
    
    # Store path for current image
    path_img = os.path.join(data_path, img_file)
    
    # Read image file
    img = cv2.imread(path_img)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Threshold image to form a binary image
    # Label the binary image and retain (crop image) the connected component with largest area
    ret,thresh = cv2.threshold(gray, 0, 255, 0)
    im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, 
                                                                        cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(c) for c in contours]
    max_index = np.argmax(areas)
    cnt=contours[max_index]
    x, y, w, h = cv2.boundingRect(cnt)
    crop_img = gray[y:y+h, x+20:x+w-10]
    
    # Histogram of image
    img_hist, bin_edges = np.histogram(crop_img, bins=256, range=(0, 255))
    
    # Divide graylevel of histogram into 3 parts (0:100; 101:200; 201:255)
    
    #hist_1 = img_hist[0:101]
    #hist_2 = img_hist[101:201]
    hist_3 = img_hist[201:256]
    
    # Find the histogram peak at each of the three divided parts
    #hist_1_peak = np.argmax(hist_1)+1
    #hist_2_peak = np.argmax(hist_2)+101
    hist_3_peak = np.argmax(hist_3)+201
    
    # Declare a binary mask (initially all zeros) of same size as the cropped image
    bin_mask = np.zeros_like(crop_img, dtype=bool)
    
    # Get all the x,y co-ordinates where the cropped image has the same intensity as the histogram peaks
    #pts_1 = np.where(crop_img == hist_1_peak)
    #pts_1 = np.transpose(np.vstack(pts_1))
    
    #pts_2 = np.where(crop_img == hist_2_peak)
    #pts_2 = np.transpose(np.vstack(pts_2))
    
    pts_3 = np.where(crop_img == hist_3_peak)
    pts_3 = np.transpose(np.vstack(pts_3))
    
    # Assign all the x,y points collected as 1 (True) in binary mask
    #for r,c in pts_1:
        #bin_mask[r, c] = True
        
    #for r,c in pts_2:
        #bin_mask[r, c] = True
    
    for r,c in pts_3:
        bin_mask[r, c] = True
        
    # Perform dilation of binary mask using a 3 X 3 structuring element
    #bin_mask_dil = morph.binary_dilation(bin_mask, selem=morph.square(3, dtype=bool))
    
    # Invert binary mask
    bin_mask_inv = np.invert(bin_mask)
    
    # Multiply binary mask with cropped image to remove predicted artifacts
    out_img = crop_img*bin_mask_inv
    
    # Restore the "gap" after removing the artifacts pixels by simulating the surrounding textures

    # Pad the image so to get inter number of 8 X 8 blocks
    r, c = out_img.shape
    r_pad = 8 - (r % 8)
    c_pad = 8 - (c % 8)
    
    r_pad_l = int(np.floor(r_pad/2))
    r_pad_r = int(np.ceil(r_pad/2))
    
    c_pad_l = int(np.floor(c_pad/2))
    c_pad_r = int(np.ceil(c_pad/2))
    
    out_img_pad = np.pad(crop_img, ((r_pad_l, r_pad_r), (c_pad_l, c_pad_r)), mode='symmetric')
    
    row_pad = out_img_pad.shape[0]
    col_pad = out_img_pad.shape[1]
    
    # Compute the two-dimensional DCT of 8-by-8 blocks in the image
    mask = [[1, 1, 1, 1, 0, 0, 0, 0],
            [1, 1, 1, 0, 0, 0, 0, 0],
            [1, 1, 0, 0, 0, 0, 0, 0],
            [1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0]]
            
    idct_img = np.zeros_like(out_img_pad)
    
    for row in range(0, row_pad-1, 8):
        for col in range(0, col_pad-1, 8):
            img_patch = out_img_pad[row:row+8, col:col+8]
            dct_img_patch = dctn(img_patch)
            
            # Discard all but 10 of the 64 DCT coefficients in each block and compute the Inverse DCT
            dct_img_patch = dct_img_patch*mask
            
            # Inverse DCT
            idct_img[row:row+8, col:col+8] = idctn(dct_img_patch)
            
    # Reduce the Inverse DCT image to same size as the original
    idct_img = idct_img[r_pad_l:row_pad-r_pad_r, c_pad_l:col_pad-c_pad_r]
    
    # Only the "gaps" are replaced by the approximated textures, preserve the original textures
    corr_img = (idct_img*bin_mask)+out_img
    
    # Resize the corrected image
    corr_res_img = cv2.resize(corr_img, (int((max_tick_bar_dist_px/tick_bar_dist[i])*corr_img.shape[0]), 
                                             corr_img.shape[1]), interpolation = cv2.INTER_CUBIC)
    i = i+1
    
    # Save the processed image
    save_img_path = os.path.join(data_processed_path, img_file)
    cv2.imwrite(save_img_path, corr_res_img)

In [6]:
# Process images without calliper marker info

# Define paths to data folder and the processed data folder
data_path = os.path.join(".", "Data_without_calliper_info")
data_processed_path = os.path.join(".", "Processed")

for img_file in sorted(os.listdir(data_path)):
    # Ignore image files that aren't .jpg files (images)
    if img_file[-4:] != '.tif':
        continue
    
    # Store path for current image
    path_img = os.path.join(data_path, img_file)
    
    # Read image file
    img = cv2.imread(path_img)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Threshold image to form a binary image
    # Label the binary image and retain (crop image) the connected component with largest area
    ret,thresh = cv2.threshold(gray, 0, 255, 0)
    im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, 
                                                                        cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(c) for c in contours]
    max_index = np.argmax(areas)
    cnt=contours[max_index]
    x, y, w, h = cv2.boundingRect(cnt)
    crop_img = gray[y:y+h, x+20:x+w-10]
    
    # Histogram of image
    img_hist, bin_edges = np.histogram(crop_img, bins=256, range=(0, 255))
    
    # Divide graylevel of histogram into 3 parts (0:100; 101:200; 201:255)
    
    #hist_1 = img_hist[0:101]
    #hist_2 = img_hist[101:201]
    hist_3 = img_hist[201:256]
    
    # Find the histogram peak at each of the three divided parts
    #hist_1_peak = np.argmax(hist_1)+1
    #hist_2_peak = np.argmax(hist_2)+101
    hist_3_peak = np.argmax(hist_3)+201
    
    # Declare a binary mask (initially all zeros) of same size as the cropped image
    bin_mask = np.zeros_like(crop_img, dtype=bool)
    
    # Get all the x,y co-ordinates where the cropped image has the same intensity as the histogram peaks
    #pts_1 = np.where(crop_img == hist_1_peak)
    #pts_1 = np.transpose(np.vstack(pts_1))
    
    #pts_2 = np.where(crop_img == hist_2_peak)
    #pts_2 = np.transpose(np.vstack(pts_2))
    
    pts_3 = np.where(crop_img == hist_3_peak)
    pts_3 = np.transpose(np.vstack(pts_3))
    
    # Assign all the x,y points collected as 1 (True) in binary mask
    #for r,c in pts_1:
        #bin_mask[r, c] = True
        
    #for r,c in pts_2:
        #bin_mask[r, c] = True
    
    for r,c in pts_3:
        bin_mask[r, c] = True
        
    # Perform dilation of binary mask using a 3 X 3 structuring element
    #bin_mask_dil = morph.binary_dilation(bin_mask, selem=morph.square(3, dtype=bool))
    
    # Invert binary mask
    bin_mask_inv = np.invert(bin_mask)
    
    # Multiply binary mask with cropped image to remove predicted artifacts
    out_img = crop_img*bin_mask_inv
    
    # Restore the "gap" after removing the artifacts pixels by simulating the surrounding textures

    # Pad the image so to get inter number of 8 X 8 blocks
    r, c = out_img.shape
    r_pad = 8 - (r % 8)
    c_pad = 8 - (c % 8)
    
    r_pad_l = int(np.floor(r_pad/2))
    r_pad_r = int(np.ceil(r_pad/2))
    
    c_pad_l = int(np.floor(c_pad/2))
    c_pad_r = int(np.ceil(c_pad/2))
    
    out_img_pad = np.pad(crop_img, ((r_pad_l, r_pad_r), (c_pad_l, c_pad_r)), mode='symmetric')
    
    row_pad = out_img_pad.shape[0]
    col_pad = out_img_pad.shape[1]
    
    # Compute the two-dimensional DCT of 8-by-8 blocks in the image
    mask = [[1, 1, 1, 1, 0, 0, 0, 0],
            [1, 1, 1, 0, 0, 0, 0, 0],
            [1, 1, 0, 0, 0, 0, 0, 0],
            [1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0]]
            
    idct_img = np.zeros_like(out_img_pad)
    
    for row in range(0, row_pad-1, 8):
        for col in range(0, col_pad-1, 8):
            img_patch = out_img_pad[row:row+8, col:col+8]
            dct_img_patch = dctn(img_patch)
            
            # Discard all but 10 of the 64 DCT coefficients in each block and compute the Inverse DCT
            dct_img_patch = dct_img_patch*mask
            
            # Inverse DCT
            idct_img[row:row+8, col:col+8] = idctn(dct_img_patch)
            
    # Reduce the Inverse DCT image to same size as the original
    idct_img = idct_img[r_pad_l:row_pad-r_pad_r, c_pad_l:col_pad-c_pad_r]
    
    # Only the "gaps" are replaced by the approximated textures, preserve the original textures
    corr_img = (idct_img*bin_mask)+out_img
    
    # Resize the corrected image
    corr_res_img = cv2.resize(corr_img, (300, 250), interpolation = cv2.INTER_CUBIC)
    
    # Save the processed image
    save_img_path = os.path.join(data_processed_path, img_file)
    cv2.imwrite(save_img_path, corr_res_img)